In [ ]:
import sys
import os

# Assuming the notebook is in 'app/lab' and the module is in 'app/app'
module_path = os.path.abspath(os.path.join('..', 'app'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from ingest import ingest

In [ ]:
ingest.teste()

In [ ]:
from unstructured.partition.auto import partition
from unstructured.chunking.title import chunk_by_title
from langchain_core.documents import Document


In [ ]:
elements = partition("../../docs/Cadastramento inicial no PJe.pdf")

In [ ]:
elements

In [ ]:
print("\n".join([str(el) for el in elements]))

In [ ]:
chunks = chunk_by_title(elements, max_characters=1500, new_after_n_chars=1000, overlap=150)
chunks

In [ ]:
chunks[1].metadata.to_dict()

In [ ]:
for chunk in chunks:
    print(chunk)
    print("\n" + "-"*80)
    # input()

In [ ]:
documents = []
for chunk in chunks:
    metadata = chunk.metadata.to_dict()
    del metadata["languages"]
    metadata["source"] = metadata["filename"]
    documents.append(Document(page_content=chunk.text, metadata=metadata))

In [ ]:
path, dirs, files = next(os.walk(path))
files

In [ ]:
documents = []
for file in files:
    with open( os.path.join(path, file), 'rb') as f:
        print(f'Opening file: {file}')
        
        # open file, clean, segment
        elements = partition(file=f)
        
        # create chunks to better rag
        chunks = chunk_by_title(elements, max_characters=1500, new_after_n_chars=1000, overlap=150)
        
        # convert to the Langchain format
        for chunk in chunks:
            metadata = chunk.metadata.to_dict()
            # print("---------------------")
            # print(metadata)
            # print("---------------------")
            # print(chunk.text)

            del metadata["languages"]
            if "filename" in metadata:
                metadata["source"] = metadata["filename"]
            else:
                metadata["source"] = file

            documents.append(Document(page_content=chunk.text, metadata=metadata))


In [ ]:
documents

In [ ]:
from langchain_community.embeddings import GPT4AllEmbeddings
model_name = "nomic-embed-text-v1.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}
embeddings = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
)

In [ ]:
import weaviate
# from langchain_weaviate.vectorstores import WeaviateVectorStore
from langchain_community.vectorstores import Weaviate

In [ ]:
index_name = "index1"
url = 'host.docker.internal'

In [ ]:
vectorstore = Weaviate.from_documents(
    documents=all_splits, embedding=OpenAIEmbeddings(), index_name=WEAVIATE_INDEX_NAME
)
# retriever = vectorstore.as_retriever()

# vectorstore = Weaviate.from_existing_index(WEAVIATE_INDEX_NAME, OpenAIEmbeddings())
# retriever = vectorstore.as_retriever()

In [ ]:


# weaviate_client = weaviate.connect_to_custom(
#     http_host=url,
#     http_port=8080,
#     http_secure=False,
#     grpc_host=url,
#     grpc_port=50051,
#     grpc_secure=False,
#     # auth_credentials=AuthApiKey(weaviate_key),   # `weaviate_key`: your Weaviate API key
# )

weaviate_client = weaviate.connect_to_local(url)
# vector_store = Weaviate(weaviate_client, index_name, "text_key")

# vector_store = WeaviateVectorStore(client=weaviate_client, index_name, "text_key")
# db = WeaviateVectorStore.from_documents(documents, embeddings, client=weaviate_client)
db = WeaviateVectorStore.from_documents([], embeddings, client=weaviate_client)

In [ ]:
vector_store.from_documents(documents, embeddings)